# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install BeautifulSoup4
!pip install lxml
!pip install geocoder

     |████████████████████████████████| 112kB 7.2MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 8.5MB/s eta 0:00:01


In [2]:
#importing libraries
import numpy as np
import pandas as pd
import json

import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

import geocoder

from sklearn.cluster import KMeans

### Step1: Getting Data

In [3]:
#scraping data from wiki website
wiki="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
website_data = requests.get(wiki).text
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(website_data, "lxml")
#print(soup.prettify())

In [4]:
#use the 'find_all' function to bring back all instances of the 'table' tag in the HTML
all_tables =soup.find_all('table')
#print(all_tables)

right_table =soup.find('table', class_='wikitable sortable')
#print(right_table)

In [35]:
#select and save info into list from html format 
A=[]
B=[]
C=[]

for row in right_table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) ==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True).rstrip())


In [36]:
#transfer to dataframe format
df =pd.DataFrame(data=A, columns=['Postcode'])
df['Borough']=B
df['Neighbourhood']=C
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Step2: Data Cleanup

In [37]:
#remove row with unassigned Borough
df=df[df['Borough'] != 'Not assigned']

#let unassigned Neighbourhood = its' Borough
df['Neighbourhood'].replace('Not assigned\n', df['Borough'], inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [38]:
df=df.groupby(["Postcode","Borough"])['Neighbourhood'].apply(lambda x: ', '.join(x)).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [39]:
df.shape

(103, 3)